In [15]:
# Derived from keras-rl
import opensim as osim
import numpy as np
import sys

from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Input, merge
from keras.optimizers import Adam

import numpy as np

from rl.agents import DDPGAgent
from rl.memory import SequentialMemory
from rl.random import OrnsteinUhlenbeckProcess

from osim.env import *

from keras.optimizers import RMSprop

import argparse
import math

print("End of importation")

End of importation


In [16]:
# Command line parameters
#parser = argparse.ArgumentParser(description='Train or test neural net motor controller')
#parser.add_argument('--train', dest='train', action='store_true', default=True)
#parser.add_argument('--test', dest='train', action='store_false', default=True)
#parser.add_argument('--steps', dest='steps', action='store', default=10000)
#parser.add_argument('--visualize', dest='visualize', action='store_true', default=False)
#parser.add_argument('--model', dest='model', action='store', default="example.h5f")
#parser.add_argument('-f', dest='kernel', action='store', default="e")
#args = parser.parse_args()

print("Arguments...")
#print(args)

args = argparse.Namespace(model='example.h5f', steps=10000, visualize=False)
print(args)
# Load walking environment
env = ArmEnv(args.visualize)
print("Environment generated")


Arguments...
Namespace(model='example.h5f', steps=10000, visualize=False)
Environment generated


## Param Debug

In [17]:
print(env)
print(env.timestep_limit)

<ArmEnv instance>
500


In [18]:
nb_actions = env.action_space.shape[0]

# Total number of steps in training
nallsteps = args.steps

# Create networks for DDPG
# Next, we build a very simple model.
actor = Sequential()
actor.add(Flatten(input_shape=(1,) + env.observation_space.shape))
actor.add(Dense(32))
actor.add(Activation('relu'))
actor.add(Dense(32))
actor.add(Activation('relu'))
actor.add(Dense(32))
actor.add(Activation('relu'))
actor.add(Dense(nb_actions))
actor.add(Activation('sigmoid'))
print(actor.summary())

action_input = Input(shape=(nb_actions,), name='action_input')
observation_input = Input(shape=(1,) + env.observation_space.shape, name='observation_input')
flattened_observation = Flatten()(observation_input)
x = merge([action_input, flattened_observation], mode='concat')
x = Dense(64)(x)
x = Activation('relu')(x)
x = Dense(64)(x)
x = Activation('relu')(x)
x = Dense(64)(x)
x = Activation('relu')(x)
x = Dense(1)(x)
x = Activation('linear')(x)
critic = Model(input=[action_input, observation_input], output=x)
print(critic.summary())

# Set up the agent for training
#warning: a too high limit gives errors
memory = SequentialMemory(limit=100000, window_length=1)
random_process = OrnsteinUhlenbeckProcess(theta=.15, mu=0., sigma=.2, size=env.noutput)
agent = DDPGAgent(nb_actions=nb_actions, actor=actor, critic=critic, critic_action_input=action_input,
                  memory=memory, nb_steps_warmup_critic=100, nb_steps_warmup_actor=100,
                  random_process=random_process, gamma=.99, target_model_update=1e-3,
                  delta_clip=1.)
#agent = ContinuousDQNAgent(nb_actions=env.noutput, V_model=V_model, L_model=L_model, mu_model=mu_model,
#                            memory=memory, nb_steps_warmup=1000, random_process=random_process,
#                            gamma=.99, target_model_update=0.1)
agent.compile(Adam(lr=.001, clipnorm=1.), metrics=['mae'])

# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
flatten_7 (Flatten)              (None, 14)            0           flatten_input_5[0][0]            
____________________________________________________________________________________________________
dense_25 (Dense)                 (None, 32)            480         flatten_7[0][0]                  
____________________________________________________________________________________________________
activation_25 (Activation)       (None, 32)            0           dense_25[0][0]                   
____________________________________________________________________________________________________
dense_26 (Dense)                 (None, 32)            1056        activation_25[0][0]              
___________________________________________________________________________________________

## Train

In [5]:
#Warning: verbose=1 freezes the notebook
agent.fit(env, nb_steps=nallsteps, visualize=False, verbose=2, nb_max_episode_steps=env.timestep_limit, log_interval=10000)
# After training is done, we save the final weights.
agent.save_weights(args.model, overwrite=True)

Training for 10000 steps ...
  500/10000: episode: 1, duration: 8.268s, episode steps: 500, steps per second: 60, episode reward: -1129.567, mean reward: -2.259 [-3.415, -0.315], mean action: 0.360 [-0.256, 1.604], mean observation: -0.368 [-36.067, 151.635], loss: 0.073267, mean_absolute_error: 0.206061, mean_q: -2.254587
 1000/10000: episode: 2, duration: 11.338s, episode steps: 500, steps per second: 44, episode reward: -216.021, mean reward: -0.432 [-1.806, -0.017], mean action: 0.315 [-0.535, 1.448], mean observation: 0.971 [-1000.000, 1000.000], loss: 1.908218, mean_absolute_error: 2.083755, mean_q: -3.617379
 1500/10000: episode: 3, duration: 14.984s, episode steps: 500, steps per second: 33, episode reward: -406.383, mean reward: -0.813 [-2.031, -0.071], mean action: 0.384 [-0.737, 1.575], mean observation: 3.036 [-1000.000, 1000.000], loss: 1.274501, mean_absolute_error: 1.453171, mean_q: -3.369452
 2000/10000: episode: 4, duration: 17.446s, episode steps: 500, steps per secon

## Test

In [6]:
agent.load_weights(args.model)
    # Finally, evaluate our algorithm for 1 episode.
agent.test(env, nb_episodes=1, visualize=False, nb_max_episode_steps=500)

Testing for 1 episodes ...
Episode 1: reward: -159.028, steps: 500
